# Generate tables in density and temperatures for the extended SCvH EOS

The original SCvH EOS table was extended to lower temperatures (and pressures?). However, these tables use $(P, T)$ as variables which is not useful for applications in hydro codes. 

We therefore have to invert $\rho \left( P, T \right)$ to $ P\left( \rho, T \right)$ by solving
$$ \rho \left( P, T=const \right) - \rho_i = 0$$
for a grid of $\rho_i$ using a root finder. The notebook uses the brent root finder and different interpolation (and extrapolation) methods.


### Import modules

In [ ]:
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy import interpolate as interp
from scipy import optimize
import numpy as np

In [ ]:
%matplotlib widget
%config InlineBackend.figure_format = 'retina'

In [ ]:
mpl.rcParams['figure.dpi'] = 100

Convert units from cgs to SI:

In [ ]:
press_unit_si = 0.1        # erg/cm^3 in Pascal
spec_energy_unit_si = 1e4  # erg/g in J/kg

In [ ]:
data = np.loadtxt("hydrogen_scvh_extended.data") 

logT_table   = data[:, 0] 
logP_table   = data[:, 1]
frac_H2      = data[:, 2]
frac_H       = data[:, 3]
logrho_table = data[:, 4]
logu_table   = data[:, 5]
logs_table   = data[:, 6]

# All the SCvH EOS tables are tabulated along isotherms
logT_table_axis = np.unique(logT_table)
nT = np.size(logT_table_axis)

print("Number of isotherms: nT = {:}".format(nT))

# The number of grid points in P are different for each isotherm
logP_table_axis = list()
logrho_isotherm = list()
logu_isotherm = list()
logs_isotherm = list()

for logT in logT_table_axis:
    logP_table_axis.append(logP_table[np.where(logT_table == logT)])
    logrho_isotherm.append(logrho_table[np.where(logT_table == logT)])
    logu_isotherm.append(logu_table[np.where(logT_table == logT)])
    logs_isotherm.append(logs_table[np.where(logT_table == logT)])

logT_min = np.min(logT_table_axis)
logT_max = np.max(logT_table_axis)

print("logT_min = {:}".format(logT_min))
print("logT_max = {:}".format(logT_max))
print()

# Mark where the original SCvH EOS table was extended below T=100K
index = np.min(np.where(logT_table_axis >= 2.0))
print("The EOS table was extended until index={:} logT={:}".format(index, logT_table_axis[index]))


In [ ]:
# Determine the min and max density of each isotherm
logrho_min = list()
logrho_max = list()

for i in range(nT):
    logrho_min.append(np.min(logrho_isotherm[i]))
    logrho_max.append(np.max(logrho_isotherm[i]))
    
    #print("i={:} logT={:} (T={:}): logrho_min={:} logrho_max={:}".format(i, logT_table_axis[i], pow(10.0, logT_table_axis[i]), logrho_min[i], logrho_max[i]))

In [ ]:
# Generate interpolation functions for each isotherm

logrho_of_logP_int = list()
logu_of_logP_int = list()
logs_of_logP_int = list()

for i in range(0, nT):
    logrho_of_logP_int.append(interp.InterpolatedUnivariateSpline(logP_table_axis[i], logrho_isotherm[i], k=1, ext='extrapolate'))
    logu_of_logP_int.append(interp.InterpolatedUnivariateSpline(logP_table_axis[i], logu_isotherm[i], k=1, ext='extrapolate'))
    logs_of_logP_int.append(interp.InterpolatedUnivariateSpline(logP_table_axis[i], logs_isotherm[i], k=1, ext='extrapolate'))
    """
    logrho_of_logP_int.append(interp.CubicSpline(logP_table_axis[i], logrho_isotherm[i], extrapolate=True))
    logu_of_logP_int.append(interp.CubicSpline(logP_table_axis[i], logu_isotherm[i], extrapolate=True))
    logs_of_logP_int.append(interp.CubicSpline(logP_table_axis[i], logs_isotherm[i], extrapolate=True))
    """
    """
    logrho_of_logP_int.append(interp.PchipInterpolator(logP_table_axis[i], logrho_isotherm[i], extrapolate=True))
    logu_of_logP_int.append(interp.PchipInterpolator(logP_table_axis[i], logu_isotherm[i], extrapolate=True))
    logs_of_logP_int.append(interp.PchipInterpolator(logP_table_axis[i], logs_isotherm[i], extrapolate=True))
    """

First plot the table and check if the result looks sensible.

In [ ]:
# Skip some curves otherwise the plots are very hard to read
nSkipT = 5
nSkipRho = 10

In [ ]:
# Plot log(rho), log(u) and log(s)
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# rho(P, T=const)
for i in range(0, nT, nSkipT):
    ax[0][0].plot(logP_table_axis[i], logrho_isotherm[i], '-')
    
ax[0][0].plot(logP_table_axis[0], logrho_isotherm[0], '--', color='black', label=r"T$_{min}$")

ax[0][0].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(rho) [g cm^${-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# Interpolated for T=const.
#plt.gca().set_prop_cycle(None)

for i in range(0, nT, nSkipT):
    ax[0][1].plot(logP_table_axis[i], logrho_isotherm[i], '-')

#plt.gca().set_prop_cycle(None)

for i in range(0, nT, nSkipT):
    logP = np.linspace(np.min(logP_table_axis[i]), np.max(logP_table_axis[i]), 1000)
    ax[0][1].plot(logP, logrho_of_logP_int[i](logP), '--')

ax[0][1].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(rho) [g cm^${-3}$]")
ax[0][1].set(title="SCvH EOS H (T=const.)")

# u(P, T=const)
for i in range(0, nT, nSkipT):
    ax[1][0].plot(logP_table_axis[i], logu_isotherm[i], '-')
    
ax[1][0].plot(logP_table_axis[0], logu_isotherm[0], '--', color='black', label=r"T$_{min}$")

ax[1][0].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# Interpolated for T=const.
for i in range(0, nT, nSkipT):
    ax[1][1].plot(logP_table_axis[i], logu_isotherm[i], '-')

for i in range(0, nT, nSkipT):
    logP = np.linspace(np.min(logP_table_axis[i]), np.max(logP_table_axis[i]), 1000)
    ax[1][1].plot(logP, logu_of_logP_int[i](logP), '--')

ax[1][1].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# s(P, T=const)
for i in range(0, nT, nSkipT):
    ax[2][0].plot(logP_table_axis[i], logs_isotherm[i], '-')
    
ax[2][0].plot(logP_table_axis[0], logs_isotherm[0], '--', color='black', label=r"T$_{min}$")

ax[2][0].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# Interpolated for T=const.
for i in range(0, nT, nSkipT):
    ax[2][1].plot(logP_table_axis[i], logs_isotherm[i], '-')

for i in range(0, nT, nSkipT):
    logP = np.linspace(np.min(logP_table_axis[i]), np.max(logP_table_axis[i]), 1000)
    ax[2][1].plot(logP, logs_of_logP_int[i](logP), '--')

ax[2][1].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
# Plot extrapolated log(rho), log(u) and log(s)
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

logP_axis_extrap = np.linspace(-10, 25)

# rho(P, T=const)
for i in range(0, nT, nSkipT):
    ax[0][0].plot(logP_table_axis[i], logrho_isotherm[i], '-')
    
ax[0][0].plot(logP_table_axis[0], logrho_isotherm[0], '--', color='black', label=r"T$_{min}$")

ax[0][0].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(rho) [g cm^${-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# Interpolated for T=const.
#plt.gca().set_prop_cycle(None)

for i in range(0, nT, nSkipT):
    ax[0][1].plot(logP_table_axis[i], logrho_isotherm[i], '-')

#plt.gca().set_prop_cycle(None)

for i in range(0, nT, nSkipT):
    logP = np.linspace(np.min(logP_table_axis[i]), np.max(logP_table_axis[i]), 1000)
    ax[0][1].plot(logP_axis_extrap, logrho_of_logP_int[i](logP_axis_extrap), '--')

ax[0][1].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(rho) [g cm^${-3}$]")
ax[0][1].set(title="SCvH EOS H (T=const.)")

# u(P, T=const)
for i in range(0, nT, nSkipT):
    ax[1][0].plot(logP_table_axis[i], logu_isotherm[i], '-')
    
ax[1][0].plot(logP_table_axis[0], logu_isotherm[0], '--', color='black', label=r"T$_{min}$")

ax[1][0].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# Interpolated for T=const.
for i in range(0, nT, nSkipT):
    ax[1][1].plot(logP_table_axis[i], logu_isotherm[i], '-')

for i in range(0, nT, nSkipT):
    logP = np.linspace(np.min(logP_table_axis[i]), np.max(logP_table_axis[i]), 1000)
    ax[1][1].plot(logP_axis_extrap, logu_of_logP_int[i](logP_axis_extrap), '--')

ax[1][1].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# s(P, T=const)
for i in range(0, nT, nSkipT):
    ax[2][0].plot(logP_table_axis[i], logs_isotherm[i], '-')
    
ax[2][0].plot(logP_table_axis[0], logs_isotherm[0], '--', color='black', label=r"T$_{min}$")

ax[2][0].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# Interpolated for T=const.
for i in range(0, nT, nSkipT):
    ax[2][1].plot(logP_table_axis[i], logs_isotherm[i], '-')

for i in range(0, nT, nSkipT):
    logP = np.linspace(np.min(logP_table_axis[i]), np.max(logP_table_axis[i]), 1000)
    ax[2][1].plot(logP_axis_extrap, logs_of_logP_int[i](logP_axis_extrap), '--')

ax[2][1].set(xlabel="log(P) [erg cm$^{-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
def func_logrho(logP, logrho_of_logP_int, logrho_i):
    """
    Calculate logrho(logP, logT=const) - logrho_i.
    """
    return logrho_of_logP_int(logP)-logrho_i

In [ ]:
# Test if root finding works
logrho_of_logP = logrho_of_logP_int[0]
logrho_int = logrho_isotherm[0][1]

logP_min = np.min(logP_table_axis[0])
logP_max = np.max(logP_table_axis[0])

print(logP_min)
print(logP_max)



sol = optimize.root_scalar(func_logrho, args=(logrho_of_logP_int[0], logrho_isotherm[0][1]), bracket=[-100, 100], method='brentq')

logP_of_logrho = sol.root

print("rho= {:}: P= {:} err= {:}".format(logrho_int, logP_of_logrho, (logP_of_logrho-logP_table_axis[0][1])/logP_of_logrho))
#sol = optimize.root_scalar(func_logrho, args=(logrho_of_logP, logrho_int), bracket=[0, 100], method='brentq')

In [ ]:
# Invert an isotherm from logrho(logP) to logP(logrho)
def inv_isotherm_logrho(logrho_axis, logrho_of_logP_int, logP_min=-100, logP_max=100):
    nRho = np.size(logrho_axis)
    
    logP_of_logrho = np.zeros(nRho)
    
    # This is shitty but it seems that root_scalar does not work with an array of inputs
    for i in range(nRho):
        sol = optimize.root_scalar(func_logrho, args=(logrho_of_logP_int, logrho_axis[i]), bracket=[logP_min, logP_max], method='brentq')
        logP_of_logrho[i] = sol.root
        
    return logP_of_logrho

In [ ]:
# Test the function inv_isotherm_logrho()
index = 0

logrho_of_logP = logrho_of_logP_int[index]
logrho_int = logrho_isotherm[index][1]

logP_min = np.min(logP_table_axis[index])
logP_max = np.max(logP_table_axis[index])

logrho_min = np.min(logrho_isotherm[index])
logrho_max = np.max(logrho_isotherm[index])

print("logP_min= {:} logP_max= {:}".format(logP_min, logP_max))
print("logrho_min= {:} logrho_max= {:}".format(logrho_min, logrho_max))


logrho_axis = np.linspace(logrho_min, logrho_max, 100)

logP_isotherm = inv_isotherm_logrho(logrho_axis, logrho_of_logP)

"""
print("logrho_axis= {:}".format(logrho_axis))
print()
print("logP={:}".format(logP_isotherm))
"""

In [ ]:
# Write the inverted EOS table to an output file
def write_eos_table(filename, logrho_axis, logT_axis, logP, logu, logs, delimiter=",", comments="#"):
    nRho = np.size(logrho_axis)
    nT = np.size(logT_axis)
    
    # Store the different isotherms in 1d arrays
    logrho = np.tile(logrho_axis, nT)
    logT = np.repeat(logT_axis, nRho)

    #logP = logP_array.flatten(order='C')
    #logu = logu_array.flatten(order='C')
    #logs = logs_array.flatten(order='C')   

    header = "nT = {:} nRho= {:}\n"\
             "{:},{:},{:},{:},{:}".format(nT, nRho, "logT [K]", "logRho [g/cc]", "logP [barye]", "logE [erg/g]", "logS [erg/g/K]")
    
    np.savetxt(filename, np.column_stack([logT, logrho, logP, logu, logs]), header=header, fmt='%15.8e', delimiter=delimiter, comments=comments)
    

In [ ]:
# Invert logrho(logP, logT) for a given logrho_axis
logrho_axis_min = -15.0
logrho_axis_max = 2.0

logrho_axis = np.linspace(logrho_axis_min, logrho_axis_max)

logP_of_logrho = list()
logu_of_logrho = list()
logs_of_logrho = list()

# Invert each isotherm
for i in range(nT):
    logP_of_logrho.append(inv_isotherm_logrho(logrho_axis, logrho_of_logP_int[i]))
    logu_of_logrho.append(logu_of_logP_int[i](logP_of_logrho[i]))
    logs_of_logrho.append(logs_of_logP_int[i](logP_of_logrho[i]))

    logP_min = np.min(logP_table_axis[index])
    logP_max = np.max(logP_table_axis[index])

    #print("Isotherm {:} (logT={:.2f}):".format(i, logT_table_axis[i]))
    #print("logP_min= {:} logP_max= {:}".format(logP_min, logP_max))
    #print("P(rho)={:}".format(logP_of_logrho[i]))

# Generate 1D numpy arrays
logP_of_logrho_1d = np.concatenate(logP_of_logrho)
logu_of_logrho_1d = np.concatenate(logu_of_logrho)
logs_of_logrho_1d = np.concatenate(logs_of_logrho)

# Generate 2D numpy arrays because these are more convenient to use
logP_array = np.stack(logP_of_logrho)
logu_array = np.stack(logu_of_logrho)
logs_array = np.stack(logs_of_logrho)

write_eos_table("hydrogen_scvh_extended.csv", logrho_axis, logT_table_axis, logP_of_logrho_1d, logu_of_logrho_1d, logs_of_logrho_1d)


In [ ]:
# Load an EOS table assuming that the variables are (rho, T) and that each isotherm has the same number of gridpoints
def load_eos_table(filename, delimiter=",", skiprows=0):
    data = np.loadtxt(filename, delimiter=delimiter, skiprows=skiprows)

    logT_table   = data[:, 0]
    logrho_table = data[:, 1]
    logP_table   = data[:, 2]
    logu_table   = data[:, 3]
    logs_table   = data[:, 4]

    
    # All the SCvH EOS tables are tabulated along isotherms
    #logT_table_axis = np.unique(logT_table)
    #nT = np.size(logT_table_axis)
    
    # All SCVH EOS tables obtained from Ravit have the same size
    nRho = 201
    nT   = 100

    logrho_table = logrho_table[0:nRho]
    logT_table = logT_table[0:np.size(logT_table):nRho]

    logrho_min = np.min(logrho_table)
    logrho_max = np.max(logrho_table)
    logT_min   = np.min(logT_table)
    logT_max   = np.max(logT_table)
    
    dlogrho = logrho_table[1:]-logrho_table[:-1]
    dlogT = logT_table[1:]-logT_table[:-1]
    
    print("Read EOS table: {:}".format(filename))
    print("logrho_min = {:}".format(logrho_min))
    print("logrho_max = {:}".format(logrho_max))
    print("logT_min   = {:}".format(logT_min))
    print("logT_max   = {:}".format(logT_max))
    print()

    # Mark where the original SCvH EOS table was extended below T=100K
    #index = np.min(np.where(logT_table>=2.0))
    #print("The EOS table was extended until index={:} logT={:}".format(index, logT_table[index]))


    # Split into arrays of constant T
    logP_array = np.split(logP_table, nT)
    logu_array = np.split(logu_table, nT)
    logs_array = np.split(logs_table, nT)
    
    # Generate 2d arrays
    logP = np.vstack(logP_array)
    logu = np.vstack(logu_array)
    logs = np.vstack(logs_array)
    
    # Generate 2D interpolation functions for the old EOS tables
    logP_int = interp.interp2d(logrho_table, logT_table, logP, kind='linear')
    logu_int = interp.interp2d(logrho_table, logT_table, logu, kind='linear')
    logs_int = interp.interp2d(logrho_table, logT_table, logs, kind='linear')

    eos_table = {
        "nRho":     nRho,
        "nT":       nT,
        "logrho":   logrho_table,
        "logT":     logT_table,
        "logP":     logP,
        "logu":     logu,
        "logs":     logs,
        "logP_int": logP_int,
        "logu_int": logu_int,
        "logs_int": logs_int,
    }
    
    return eos_table

In [ ]:
# Test if reading the old EOS tables work
scvheos_h_old = load_eos_table("scvh_h_dt_cgs.csv", skiprows=1)

# Plot curves of log(P), log(u) and log(s)
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)


# P(rho, T=const)
for i in range(0, scvheos_h_old["nT"], nSkipT):
    ax[0][0].plot(scvheos_h_old["logrho"], scvheos_h_old["logP"][i,:], '-')
    
ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, scvheos_h_old["nRho"], nSkipRho):
    ax[0][1].plot(scvheos_h_old["logT"], scvheos_h_old["logP"][:,i], '-')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, scvheos_h_old["nT"], nSkipT):
    ax[1][0].plot(scvheos_h_old["logrho"], scvheos_h_old["logu"][i,:], '-')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, scvheos_h_old["nRho"], nSkipRho):
    ax[1][1].plot(scvheos_h_old["logT"], scvheos_h_old["logu"][:,i], '-')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, scvheos_h_old["nT"], nSkipT):
    ax[2][0].plot(scvheos_h_old["logrho"], scvheos_h_old["logs"][i,:], '-')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, scvheos_h_old["nRho"], nSkipRho):
    ax[2][1].plot(scvheos_h_old["logT"], scvheos_h_old["logs"][:,i], '-')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
# Compare the inverted EOS table to the tables I obtained from Ravit
scvheos_h_old = load_eos_table("scvh_h_dt_cgs.csv", skiprows=1)

# Generate 2D interpolation functions for the new EOS tables
logP_int = interp.interp2d(logrho_axis, logT_table_axis, logP_array, kind='linear')
logu_int = interp.interp2d(logrho_axis, logT_table_axis, logu_array, kind='linear')
logs_int = interp.interp2d(logrho_axis, logT_table_axis, logs_array, kind='linear')

# Plot curves of log(P), log(u) and log(s)
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# P(rho, T=const)
for i in range(0, scvheos_h_old["nT"], nSkipT):
    ax[0][0].plot(scvheos_h_old["logrho"], scvheos_h_old["logP"][i,:], '-')
    ax[0][0].plot(scvheos_h_old["logrho"], logP_int(scvheos_h_old["logrho"], scvheos_h_old["logT"][i]), '--')

ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, scvheos_h_old["nRho"], nSkipRho):
    ax[0][1].plot(scvheos_h_old["logT"], scvheos_h_old["logP"][:,i], '-')
    ax[0][1].plot(scvheos_h_old["logT"], logP_int(scvheos_h_old["logrho"][i], scvheos_h_old["logT"]), '--')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, scvheos_h_old["nT"], nSkipT):
    ax[1][0].plot(scvheos_h_old["logrho"], scvheos_h_old["logu"][i,:], '-')
    ax[1][0].plot(scvheos_h_old["logrho"], logu_int(scvheos_h_old["logrho"], scvheos_h_old["logT"][i]), '--')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, scvheos_h_old["nRho"], nSkipRho):
    ax[1][1].plot(scvheos_h_old["logT"], scvheos_h_old["logu"][:,i], '-')
    ax[1][1].plot(scvheos_h_old["logT"], logu_int(scvheos_h_old["logrho"][i], scvheos_h_old["logT"]), '--')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, scvheos_h_old["nT"], nSkipT):
    ax[2][0].plot(scvheos_h_old["logrho"], scvheos_h_old["logs"][i,:], '-')
    ax[2][0].plot(scvheos_h_old["logrho"], logs_int(scvheos_h_old["logrho"], scvheos_h_old["logT"][i]), '--')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, scvheos_h_old["nRho"], nSkipRho):
    ax[2][1].plot(scvheos_h_old["logT"], scvheos_h_old["logs"][:,i], '-')
    ax[2][1].plot(scvheos_h_old["logT"], logs_int(scvheos_h_old["logrho"][i], scvheos_h_old["logT"]), '--')


ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
# Generate a restricted EOS table
scvheos_h_old = load_eos_table("scvh_h_dt_cgs.csv", skiprows=1)

# Limit the data to the region in which we are interested
logrho_grid_min = np.min(scvheos_h_old['logrho'])
logrho_grid_max = -4.0

logT_grid_min = np.min(scvheos_h_old['logT'])
logT_grid_max = 3.49

# Remove the last isotherm because it causes problems when extrapolating
logT_grid_max = 3.46

print("logrho_grid_min= {:} rho_grid_min= {:}".format(logrho_grid_min, 10**logrho_grid_min))
print("logrho_grid_max= {:} rho_grid_max= {:}".format(logrho_grid_max, 10**logrho_grid_max))
print("logT_grid_min= {:} T_grid_min= {:}".format(logT_grid_min, 10**logT_grid_min))
print("logT_grid_max= {:} T_grid_max= {:}".format(logT_grid_max, 10**logT_grid_max))
print()

index_logrho_grid = np.where(scvheos_h_old['logrho'] < logrho_grid_max)[0]
index_logT_grid = np.where(scvheos_h_old['logT'] < logT_grid_max)[0]

nRho_grid = np.size(index_logrho_grid)
nT_grid = np.size(index_logT_grid)
print("nRho= {:} nT= {:}".format(nRho_grid, nT_grid))

logrho_grid = scvheos_h_old['logrho'][index_logrho_grid]
logT_grid = scvheos_h_old['logT'][index_logT_grid]

logP_grid = scvheos_h_old['logP'][:nT_grid,:nRho_grid]
logu_grid = scvheos_h_old['logu'][:nT_grid,:nRho_grid]
logs_grid = scvheos_h_old['logs'][:nT_grid,:nRho_grid]

print(logP.shape)
print(logP_grid.shape)

In [ ]:
# Compare the inverted EOS table to the restricted EOS tables
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

# P(rho, T=const)
for i in range(0, nT_grid, nSkipT):
    ax[0][0].plot(logrho_grid, logP_grid[i,:], '-')
    ax[0][0].plot(logrho_grid, logP_int(logrho_grid, logT_grid[i]), '--')

ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, nRho_grid, nSkipRho):
    ax[0][1].plot(logT_grid, logP_grid[:,i], '-')
    ax[0][1].plot(logT_grid, logP_int(logrho_grid[i], logT_grid), '--')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, nT_grid, nSkipT):
    ax[1][0].plot(logrho_grid, logu_grid[i,:], '-')
    ax[1][0].plot(logrho_grid, logu_int(logrho_grid, logT_grid[i]), '--')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, nRho_grid, nSkipRho):
    ax[1][1].plot(logT_grid, logu_grid[:,i], '-')
    ax[1][1].plot(logT_grid, logu_int(logrho_grid[i], logT_grid), '--')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, nT_grid, nSkipT):
    ax[2][0].plot(logrho_grid, logs_grid[i,:], '-')
    ax[2][0].plot(logrho_grid, logs_int(logrho_grid, logT_grid[i]), '--')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, nRho_grid, nSkipRho):
    ax[2][1].plot(logT_grid, logs_grid[:,i], '-')
    ax[2][1].plot(logT_grid, logs_int(logrho_grid[i], logT_grid), '--')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")


In [ ]:
# Compare the inverted EOS table to the restricted EOS tables when extrapolating
fig, ax = plt.subplots(3, 2)

x, y = fig.get_size_inches()

fig.set_size_inches(2*x, 3*y)

nT_ext = np.size(logT_table_axis)
nRho_ext = np.size(logrho_axis)

# Isotherms
logP_of_logrho_grid_int = list()
logu_of_logrho_grid_int = list()
logs_of_logrho_grid_int = list()

logP_of_logrho_ext_int = list()
logu_of_logrho_ext_int = list()
logs_of_logrho_ext_int = list()

for i in range(0, nT_grid):
    logP_of_logrho_grid_int.append(interp.InterpolatedUnivariateSpline(logrho_grid, logP_grid[i,:], k=1, ext='extrapolate'))
    logu_of_logrho_grid_int.append(interp.InterpolatedUnivariateSpline(logrho_grid, logu_grid[i,:], k=1, ext='extrapolate'))
    logs_of_logrho_grid_int.append(interp.InterpolatedUnivariateSpline(logrho_grid, logs_grid[i,:], k=1, ext='extrapolate'))

for i in range(nT_ext):
    logP_of_logrho_ext_int.append(interp.InterpolatedUnivariateSpline(logrho_axis, logP_array[i,:], k=1, ext='extrapolate'))
    logu_of_logrho_ext_int.append(interp.InterpolatedUnivariateSpline(logrho_axis, logu_array[i,:], k=1, ext='extrapolate'))
    logs_of_logrho_ext_int.append(interp.InterpolatedUnivariateSpline(logrho_axis, logs_array[i,:], k=1, ext='extrapolate'))

# rho=const.
logP_of_logT_grid_int = list()
logu_of_logT_grid_int = list()
logs_of_logT_grid_int = list()

logP_of_logT_ext_int = list()
logu_of_logT_ext_int = list()
logs_of_logT_ext_int = list()

for i in range(0, nRho_grid):
    logP_of_logT_grid_int.append(interp.InterpolatedUnivariateSpline(logT_grid, logP_grid[:,i], k=1, ext='extrapolate'))
    logu_of_logT_grid_int.append(interp.InterpolatedUnivariateSpline(logT_grid, logu_grid[:,i], k=1, ext='extrapolate'))
    logs_of_logT_grid_int.append(interp.InterpolatedUnivariateSpline(logT_grid, logs_grid[:,i], k=1, ext='extrapolate'))

for i in range(0, nRho_ext):
    logP_of_logT_ext_int.append(interp.InterpolatedUnivariateSpline(logT_table_axis, logP_grid[:,i], k=1, ext='extrapolate'))
    logu_of_logT_ext_int.append(interp.InterpolatedUnivariateSpline(logT_table_axislogT_grid, logu_grid[:,i], k=1, ext='extrapolate'))
    logs_of_logT_ext_int.append(interp.InterpolatedUnivariateSpline(logT_grid, logs_grid[:,i], k=1, ext='extrapolate'))

logrho_axis_extrap = np.concatenate((np.linspace(-15.0, logrho_grid_min, 10), logrho_grid))
logT_axis_extrap = np.concatenate((np.linspace(0.1, logT_grid_min, 10), logT_grid))

nRho_extrap = np.size(logrho_axis_extrap)
nT_extrap = np.size(logT_axis_extrap)

print("nRho_extrap= {:} nRho_grid= {:}".format(nRho_extrap, nRho_grid))
print("nT_extrap= {:} nT_grid= {:}".format(nT_extrap, nT_grid))

#logrho_axis_extrap = np.linspace(-15.0, -4.0, nRho_grid)
#logT_axis_extrap = np.linspace(0.0, 3, nT_grid)

#logrho_axis_extrap = logrho_grid
#logT_axis_extrap = logT_grid
    
# P(rho, T=const)
for i in range(0, nT_grid, nSkipT):
    ax[0][0].plot(logrho_axis_extrap, logP_of_logrho_grid_int[i](logrho_axis_extrap), '-')
    
for i in range(0, nT_extrap, nSkipT):
    ax[0][0].plot(logrho_axis_extrap, logP_int(logrho_axis_extrap, logT_axis_extrap[i]), '--')

ax[0][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][0].set(title="SCvH EOS H (T=const.)")

# P(rho=const, T)
for i in range(0, nRho_extrap, nSkipRho):
    ax[0][1].plot(logT_axis_extrap, logP_of_logT_grid_int[i](logT_axis_extrap), '-')
    ax[0][1].plot(logT_axis_extrap, logP_int(logrho_axis_extrap[i], logT_axis_extrap), '--')

ax[0][1].set(xlabel="log(T) [K]", ylabel="log(P) [erg cm$^{-3}$]")
ax[0][1].set(title="SCvH EOS H (rho=const.)")

# u(rho, T=const)
for i in range(0, nT_extrap, nSkipT):
    ax[1][0].plot(logrho_axis_extrap, logu_of_logrho_grid_int[i](logrho_axis_extrap), '-')
    ax[1][0].plot(logrho_axis_extrap, logu_int(logrho_axis_extrap, logT_axis_extrap[i]), '--')

ax[1][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(u) [erg g$^{-1}$]")

# u(rho=const, T)
for i in range(0, nRho_extrap, nSkipRho):
    ax[1][1].plot(logT_axis_extrap, logu_of_logT_grid_int[i](logT_axis_extrap), '-')
    ax[1][1].plot(logT_axis_extrap, logu_int(logrho_axis_extrap[i], logT_axis_extrap), '--')

ax[1][1].set(xlabel="log(T) [K]", ylabel="log(u) [erg g$^{-1}$]")

# s(rho, T=const)
for i in range(0, nT_extrap, nSkipT):
    ax[2][0].plot(logrho_axis_extrap, logs_of_logrho_grid_int[i](logrho_axis_extrap), '-')
    ax[2][0].plot(logrho_axis_extrap, logs_int(logrho_axis_extrap, logT_axis_extrap[i]), '--')

ax[2][0].set(xlabel="log(rho) [g cm^${-3}$]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")

# s(rho=const, T)
for i in range(0, nRho_extrap, nSkipRho):
    ax[2][1].plot(logT_grid, logs_grid[:,i], '-')
    ax[2][1].plot(logT_grid, logs_int(logrho_grid[i], logT_grid), '--')

ax[2][1].set(xlabel="log(T) [K]", ylabel="log(s) [erg g$^{-1}$ K$^{-1}$]")
